In [12]:
import numpy as np
import netCDF4 as nc

from wrf import getvar, interplevel, to_np, latlon_coords

def read_dataset(input_file):
    input_nc = nc.Dataset(input_file)
    date = getvar(input_nc,'Times')
    time = input_nc.variables['ZNU'].shape[0]
    levels = input_nc.variables['ZNU'].shape[1]
    
    return (input_nc, date, time, levels)

def add_noise(variable, noise):
    new_variable = variable + noise * np.random.randn(variable.shape[0], variable.shape[1])
    return new_variable.astype(dtype='float32')

def perturbate_variable(variable, time, levels, noise):
    perturbated_variable = np.empty_like(variable)
    for hr in range(time):
        for lvl in range(levels):
            perturbated_variable[hr][lvl] = add_noise(variable[hr][lvl], noise)
    return perturbated_variable

def perturbate_dataset(input_nc, output_file, noise, time, levels, perturbate_var_func):
    output_nc = nc.Dataset(output_file, mode='w')
    
    # Create the dimensions of the file
    for name, dim in input_nc.dimensions.items():
        output_nc.createDimension(name, len(dim) if not dim.isunlimited() else None)
    
    # Copy the global attributes
    output_nc.setncatts({ a: input_nc.getncattr(a) for a in input_nc.ncattrs() })
    
    # Create the variables in the file
    for var_name, var in input_nc.variables.items():
        output_nc.createVariable(var_name, var.dtype, var.dimensions)

        # Copy the variable attributes
        output_nc.variables[var_name].setncatts({ a: var.getncattr(a) for a in var.ncattrs()})

        # Copy the variables values (as 'f4' eventually)
        if var_name in ['U', 'V', 'W', 'P', 'T']:
            output_nc.variables[var_name][:] = perturbate_var_func(input_nc.variables[var_name], time, levels, noise)
        else:
            output_nc.variables[var_name][:] = input_nc.variables[var_name][:]
    
    # Save and close file
    output_nc.close()

def generate_ensemble(input_file, output_dir, member_numbers=10, noise=0.01):
    input_nc, date, time, levels = read_dataset(input_file)
    
    for index in range(0, member_numbers):
        perturbate_dataset(input_nc, f"{output_dir}/{index}_wrfout_perturbed.nc", noise, time, levels, perturbate_variable)


In [13]:
generate_ensemble('./wrfout.nc', './output')

In [14]:
input_file = './wrfout.nc'
output_file = './output/0_wrfout_perturbed.nc'
nc_file = nc.Dataset(input_file)
nc_file2 = nc.Dataset(output_file)

for name in nc_file.variables.keys():
    if (nc_file2.variables[name][:]!=nc_file.variables[name][:]).any():
        print(name)

U
V
W
T
P
